In [70]:
import numpy as np
import matplotlib as plt
import matplotlib.pyplot
import matplotlib.axes
import matplotlib.axis
from matplotlib import contour

In [75]:
def database_gen():
    #puntos
    nx=100
    ny=100
    nt=100
    #dimensiones
    Lx=1.0
    Ly=1.0
    #tiempo total
    T=1.0
    #Generacion de datos
    A=1.0
    kx=2*np.pi
    ky=2*np.pi
    f_t=1.0
    phi=0.0
    sigma=0.05
    seed=2
    
    channel_first=True
    dtype=np.float32
    
    rng = np.random.default_rng(seed)
    x = np.linspace(0, Lx, nx, dtype=dtype)
    y = np.linspace(0, Ly, ny, dtype=dtype)
    t = np.linspace(0, T, nt, dtype=dtype)
    X, Y = np.meshgrid(x, y, indexing='xy')  # (ny, nx)

    u = np.empty((nt, ny, nx), dtype=dtype)
    v = np.empty((nt, ny, nx), dtype=dtype)
    
    for i, ti in enumerate(t): #bucle que recorre cada instante t 
        phase_t = 2*np.pi*f_t*ti
        u_clean = A * np.sin(phase_t) * np.cos(kx*X) * np.cos(ky*Y)
        v_clean = A * np.cos(phase_t + phi) * np.sin(kx*X) * np.sin(ky*Y)
        u[i] = u_clean + rng.normal(0.0, sigma, size=(ny, nx))
        v[i] = v_clean + rng.normal(0.0, sigma, size=(ny, nx))
    
    F = np.stack([u, v], axis=1)
    
    np.savez_compressed("campo_vel.npz", F=F)
    return F
    

La manera en la que se genera el array de la database el F(tiempo, canal, pos. y, pos. x).
Canales: 0 > u
         1 > v

In [77]:
F = database_gen()

In [83]:
F.shape
print(F[99,1,50,50])

0.05106939
